In [ ]:
import numpy as np
from queue import PriorityQueue
from utils import *
import random
from copy import deepcopy
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import matplotlib.pyplot as plt

import torch
import torch.nn.functional as F
from torch_geometric.utils import from_networkx
import networkx as nx
from torch_geometric.nn import SAGEConv, global_mean_pool
from torch_geometric.utils import negative_sampling


In [1]:
import networkx as nx

# --- Step 1: Create directed graph ---
G_original = nx.DiGraph()
G_original.add_nodes_from(["Attacker", "Data Server", "Pad", "Web Server", "Host 1", "Host 2", "Host 3", "File Server"])

# --- Step 2: Add node attributes (priority + state) ---
# Example initialization: state = 0 (normal), 1 (alert)
# priority = 1-10 (importance scale, can adjust per node)
node_attributes = {
    "Attacker":   {"state": 1, "priority": 10},  # attacker always active
    "Data Server": {"state": 0, "priority": 9},
    "Pad":        {"state": 0, "priority": 3},
    "Web Server": {"state": 0, "priority": 6},
    "Host 1":     {"state": 0, "priority": 4},
    "Host 2":     {"state": 0, "priority": 5},
    "Host 3":     {"state": 0, "priority": 5},
    "File Server": {"state": 0, "priority": 7},
}
nx.set_node_attributes(G_original, node_attributes)

# --- Step 3: Add edges with weights ---
edges = [
    ("Attacker", "Pad", {"user": 0.6, "root": 0.6}),
    ("Attacker", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Attacker", "Host 1", {"user": 0.6, "root": 0.48}),
    ("Pad", "Host 1", {"user": 0.6, "root": 0.48}),
    ("Pad", "Host 2", {"user": 0.32, "root": 0.32}),
    ("Pad", "Host 3", {"user": 0.32, "root": 0.32}),
    ("Pad", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Host 1", "Pad", {"user": 0.6, "root": 0.6}),
    ("Host 1", "Web Server", {"user": 0.8, "root": 0.6}),
    ("Host 1", "Host 2", {"user": 0.32, "root": 0.32}),
    ("Host 1", "Host 3", {"user": 0.32, "root": 0.32}),
    ("Host 2", "Host 3", {"user": 0.8, "root": 0.8}),
    ("Host 2", "File Server", {"user": 0.8, "root": 0.6}),
    ("Host 2", "Data Server", {"user": 0.8, "root": 0.6}),
    ("Host 3", "Host 2", {"user": 0.8, "root": 0.8}),
    ("Host 3", "File Server", {"user": 0.8, "root": 0.6}),
    ("Host 3", "Data Server", {"user": 0.8, "root": 0.6}),
    ("Web Server", "File Server", {"user": 0.8, "root": 0.04}),
    ("Web Server", "Data Server", {"user": 0.8, "root": 0.04}),
    ("File Server", "Data Server", {"user": 0.8, "root": 0.04})
]
G_original.add_edges_from(edges)



In [2]:
import torch
from torch_geometric.utils import from_networkx

# Convert networkx -> PyG Data
data = from_networkx(G_original)

# At this point, node attributes & edge attributes become tensors if they are numeric
print(data)


Data(edge_index=[2, 20], state=[8], priority=[8], user=[20], root=[20], num_nodes=8)


In [3]:
# --- Step 1: Convert networkx to PyG ---
# Your graph is directed, but PyG expects undirected for GraphSAGE (you can keep it directed if meaningful)
# G_undirected = G.to_undirected()

# Convert and keep edge attributes
data = from_networkx(G_original)

# --- Step 2: Add node features ---
# If you don’t have real features, use one-hot encoding (identity matrix)
num_nodes = G_original.number_of_nodes()
data.x = torch.eye(num_nodes, dtype=torch.float)

# Example: Assign label (optional, for supervised learning)
# Let's say "Data Server" = target class (label 1), others = 0
node_mapping = {n: i for i, n in enumerate(G_original.nodes())}
labels = torch.zeros(num_nodes, dtype=torch.long)
labels[node_mapping["Data Server"]] = 1
data.y = labels

In [4]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import NNConv, global_mean_pool

class EdgeGraphSAGE(torch.nn.Module):
    def __init__(self, node_in_dim, edge_in_dim, hidden_dim, embedding_dim):
        super().__init__()
        # MLP to transform edge_attr into weights for message passing
        nn1 = torch.nn.Sequential(
            torch.nn.Linear(edge_in_dim, hidden_dim * node_in_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim * node_in_dim, hidden_dim)
        )
        self.conv1 = NNConv(node_in_dim, hidden_dim, nn1, aggr='mean')

        nn2 = torch.nn.Sequential(
            torch.nn.Linear(edge_in_dim, hidden_dim * hidden_dim),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_dim * hidden_dim, embedding_dim)
        )
        self.conv2 = NNConv(hidden_dim, embedding_dim, nn2, aggr='mean')

    def forward(self, x, edge_index, edge_attr, batch):
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        graph_emb = global_mean_pool(x, batch)  # Graph-level embedding
        return graph_emb


In [7]:
# --- Step 4: Train the model ---
embedding_dim = 64
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

node_in_dim = data.x.size(1)             # node feature dimension
edge_in_dim = data.edge_attr.size(1)     # edge feature dimension

model = EdgeGraphSAGE(node_in_dim, 32, embedding_dim, edge_in_dim).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

Q = 5  # number of negative samples per positive edge

for epoch in range(100):
    model.train()
    optimizer.zero_grad()

    # Node embeddings
    z = model(data.x, data.edge_index, data.edge_attr)  # [num_nodes, embedding_dim]

    # Positive edges = real graph edges
    pos_edge_index = data.edge_index

    # Sample Q * num_pos_edges negatives
    neg_edge_index = negative_sampling(
        edge_index=pos_edge_index,
        num_nodes=data.num_nodes,
        num_neg_samples=Q * pos_edge_index.size(1),
        method='sparse'
    )

    # ---- Positive loss ----
    pos_score = (z[pos_edge_index[0]] * z[pos_edge_index[1]]).sum(dim=-1)  # dot product
    pos_loss = -torch.log(torch.sigmoid(pos_score) + 1e-15).mean()

    # ---- Negative loss ----
    neg_score = (z[neg_edge_index[0]] * z[neg_edge_index[1]]).sum(dim=-1)
    neg_loss = -torch.log(torch.sigmoid(-neg_score) + 1e-15).mean()

    # Full loss = positive + Q * negative expectation
    loss = pos_loss + Q * neg_loss   # <-- fixed here

    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f"Epoch {epoch}, Loss {loss.item():.4f}")


AttributeError: 'NoneType' object has no attribute 'size'